In [1]:
import twitter
import pandas as pd


https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

In [2]:
twitter_api = twitter.Api(consumer_key='f2ujCRaUnQJy4PoiZvhRQL4n4',
                        consumer_secret='EjBSQirf7i83T7CX90D5Qxgs9pTdpIGIsVAhHVs5uvd0iAcw5V',
                        access_token_key='1272989631404015616-5XMQkx65rKfQU87UWAh40cMf4aCzSq',
                        access_token_secret='emfWcF8fyfqoyywfPCJnz4jXt6DFXfndro59UK9IMAMgy')

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Tue Jun 16 20:29:26 +0000 2020", "default_profile": true, "default_profile_image": true, "id": 1272989631404015616, "id_str": "1272989631404015616", "name": "Nicola Osrin", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "NicolaOsrin"}


In [3]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [4]:
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:Airbnb
Fetched 100 tweets for the term Airbnb
[{'text': 'The Pics I showed of me and Selena march 9 in Austin  should make it clear that we were together that night and wen… https://t.co/uvUMHc8x2m', 'label': None}, {'text': 'Furthermore I stayed with Selena and our friends at an airbnb on the 9th and on the 10th stayed at a Westin because… https://t.co/xCBBlzRs8D', 'label': None}, {'text': 'Not really one to get involved in this stuff but I can confirm I hooped with with @justinbieber and a bunch of othe… https://t.co/7p3o8GhMik', 'label': None}, {'text': 'RT @tmcleod3: Not really one to get involved in this stuff but I can confirm I hooped with with @justinbieber and a bunch of other folks th…', 'label': None}]


In [126]:
#df = pd.DataFrame(list())
#df.to_csv('tweetDataFile.csv')

In [5]:
def buildTrainingSet(corpusFile, tweetDataFile, size):
    import csv
    import time
    
    count = 0
    corpus = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            if count <= size: 
                corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
                count += 1
            else: 
                break

    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [23]:
'''
corpusFile = "corpus.csv"
tweetDataFile = "tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile, 1000)
'''

'\ncorpusFile = "corpus.csv"\ntweetDataFile = "tweetDataFile.csv"\n\ntrainingData = buildTrainingSet(corpusFile, tweetDataFile, 1000)\n'

In [6]:
corp = pd.read_csv("corpus.csv", header = 0 , names = ['topic','label', 'tweet_id'] )
corp['label'].value_counts()

neutral       2503
irrelevant    1786
negative       654
positive       569
Name: label, dtype: int64

In [7]:
trainingData_copied = pd.read_csv("tweetDataFile.csv", header = None, names = ['tweet_id', 'text', 'label', 'topic'])
trainingData_copied.head()

,tweet_id,text,label,topic
0,126415614616154112,Now all @Apple has to do is get swype on the i...,positive,apple
1,126402758403305474,Hilarious @youtube video - guy does a duet wit...,positive,apple
2,126397179614068736,@RIM you made it too easy for me to switch to ...,positive,apple
3,126379685453119488,The 16 strangest things Siri has said so far. ...,positive,apple
4,126377656416612353,Great up close & personal event @Apple tonight...,positive,apple


In [8]:
trainingData_copied['label'].value_counts()

neutral     316
negative    250
positive    114
Name: label, dtype: int64

In [9]:
trainingData_copied = trainingData_copied.to_dict('records')

### Use NLTK library to filter for keywords and remove irrelevant words in tweets

In [10]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
#nltk.download()

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [11]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData_copied)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

### Build the vocabulary

In [12]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

### Matching tweets against our vocabulary

In [13]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

### Building our feature vector

In [14]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

### Train the Naives Bayes Classifier

In [15]:
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)


## Test Classifier 

In [22]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
    print("Number of negative comments = " + str(NBResultLabels.count('negative')))
    print("Number of positive comments = " + str(NBResultLabels.count('positive')))
    print("Number of neutral comments = " + str(NBResultLabels.count('neutral')))

Overall Negative Sentiment
Negative Sentiment Percentage = 37.0%
Positive Sentiment Percentage = 0.0%
Number of negative comments = 37
Number of positive comments = 0
Number of neutral comments = 63
